In [1]:
from bs4 import BeautifulSoup
import requests
import urllib
import os
import re
from urllib.request import urlopen

import pandas as pd
import numpy as np

In [2]:
def make_soup(url):
    '''
    INPUT: url string
    OUTPUT: BeautifulSoup object
    '''
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    return soup

In [3]:
def grab_body(url):
    '''
    INPUT: blog post url
    OUTPUT: list of sentence strings
    '''
    soup = make_soup(url)
    art = soup.article
    recipe = art.find('div', class_="wprm-recipe-container")
    recipe_start = str(recipe)[:30]
    idx = str(art).index(recipe_start)
    data = str(art)[:idx]
    data = re.sub(r'<.*?>', '', data)
    data = re.sub('\xa0', '', data)
    data = re.sub('\n', ' ', data)
    return data, soup.title.text

In [4]:
def get_bodies_on_page(url):
    df = pd.DataFrame(columns=['title', 'body', 'url'])
    links = get_recipe_links(url)
    for link in links:
        try:
            body, title = grab_body(link)
            A = pd.DataFrame([[title, body, link]], columns=['title', 'body', 'url'])
            df = pd.concat([df,A], axis=0)
        except:
            continue
    return df

In [37]:
def get_categories(url):
    soup = make_soup(url)
    cats = soup.find_all('option', class_='level-0')
    cat_names = [cat.text for cat in cats]
    # remove duplicates
    return cat_names[25:]

In [43]:
def get__recipe_links(url):
    soup = make_soup(url)
    links = [link.get("href") for link in soup.find_all('a', class_='more-link')]
    return links

- for each category
    - for each page
        - get body from each recipe on page